# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Group Project: Predicting Solar Power Generation with Weather Forecasting
By: Garrett Hart, Joseph Hicks, and Corina Lentz

---

## Weather Data Gathering
---

Since we have data on solar panel output from an array in Antwerp, Belgium, we will now require daily weather data to create the features we will use to predict solar panel output. NCEI doesn't have particularly detailed weather information on Antwerp (the nearest station is in Brussels), so instead I created a [World Weather Online](https://www.worldweatheronline.com/) account and utilized their 60 day free trial to gather the needed data. The documentation for the API can be found [here](https://www.worldweatheronline.com/developer/api/docs/historical-weather-api.aspx#qparameter). **The API documentation also includes a data dictionary.**

In [1]:
# We need a base url to start our queries
base_url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx'

In [2]:
# Imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import time

In [3]:
# Saving the credentials JSON file - this was made from a now deleted dictionary with the API key
# with open('credentials.json', 'w') as f:
#     json.dump(creds, f, ensure_ascii = False)

In [3]:
# Load the API key
# with open('credentials.json') as credentials:    
#     creds = json.load(credentials)
    
# key = creds['key']

In [4]:
# Make the request - just to get a sense of what the resulting data will look like
res = requests.get(url= base_url, params = {
    'q'        : 'Antwerp,Belgium',
    'date'     : '2011-10-26',
    'enddate'  : '2011-11-26',
    'key'      : key,
    'tp'       : '12', # This gives us weather conditions at noon and midnight
    'isDayTime': 'yes', 
    'format'   : 'json'
})

res.status_code

200

In [5]:
json = res.json()

In [6]:
df = pd.DataFrame(json['data']['weather'])

In [7]:
type((json['data']['weather']))

list

In [8]:
df.columns # This will come in handy in the function below

Index(['date', 'astronomy', 'maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', 'hourly'],
      dtype='object')

This query only captured the first 35 days of the timeframe, so I will need to make multiple requests to get all of the relevant data:

In [11]:
def weather_wrangler(sdate, edate, frequency='30D'):
    """Pulls weather data in monthly chunks and creates a single dataframe"""
    # Create the empty dataframe for all weather data to be appended to
    data = pd.DataFrame(columns=df.columns)
    
    # Create date list
    date_list = list(pd.date_range(start=sdate, end=edate, freq=frequency))
    for i, date in enumerate(date_list):
        
        # Trying to prevent the out of index error - and make the final data request
        if date_list[i + 1] == date_list[-1]:
            res = requests.get(url= base_url, params = {
                                'q'        : 'Antwerp,Belgium',
                                'date'     : date,
                                'enddate'  : edate,
                                'key'      : key,
                                'tp'       : '12',
                                'isDayTime': 'yes',
                                'format'   : 'json'
                               })
            print(res.status_code)
            json = res.json()
            chunk_df = pd.DataFrame(json['data']['weather'])
            data = data.append(json['data']['weather'])
            
            # Return the data in dataframe format
            return data
        
        # Make the get request and append to data
        res = requests.get(url= base_url, params = {
                                'q'        : 'Antwerp,Belgium',
                                'date'     : date,
                                'enddate'  : date_list[i+1],
                                'key'      : key,
                                'tp'       : '12',
                                'isDayTime': 'yes',
                                'format'   : 'json'
                          })

        print(res.status_code)
        json = res.json()
        chunk_df = pd.DataFrame(json['data']['weather'])
        data = data.append(chunk_df)
        
        # Wait before the next request to not overload the server with requests
        time.sleep(5)

In [12]:
df_2 = weather_wrangler(sdate='2011-10-26', edate='2020-11-10')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [13]:
df_2.head()

,date,astronomy,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex,hourly
0,2011-10-26,"[{'sunrise': '08:24 AM', 'sunset': '06:28 PM',...",13,55,7,44,10,50,0.0,10.0,3,"[{'time': '0', 'tempC': '9', 'tempF': '48', 'w..."
1,2011-10-27,"[{'sunrise': '08:25 AM', 'sunset': '06:26 PM',...",15,59,7,44,11,53,0.0,10.0,4,"[{'time': '0', 'tempC': '7', 'tempF': '45', 'w..."
2,2011-10-28,"[{'sunrise': '08:27 AM', 'sunset': '06:24 PM',...",17,63,10,50,14,57,0.0,9.9,4,"[{'time': '0', 'tempC': '10', 'tempF': '50', '..."
3,2011-10-29,"[{'sunrise': '08:29 AM', 'sunset': '06:22 PM',...",17,63,10,50,14,58,0.0,6.6,3,"[{'time': '0', 'tempC': '11', 'tempF': '52', '..."
4,2011-10-30,"[{'sunrise': '08:31 AM', 'sunset': '06:20 PM',...",15,60,10,51,13,56,0.0,8.3,3,"[{'time': '0', 'tempC': '12', 'tempF': '54', '..."


There appears to be a lot more information on hourly weather in the `hourly` column, but the data here is formatted as a list of dictionaries. To get a better look at the data in this column:

In [14]:
hourly_series = df_2['hourly']

In [15]:
hourly_series = hourly_series.to_list()

In [16]:
hourly_series[2][1]

{'time': '1200',
 'tempC': '17',
 'tempF': '62',
 'windspeedMiles': '1',
 'windspeedKmph': '1',
 'winddirDegree': '185',
 'winddir16Point': 'S',
 'weatherCode': '116',
 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}],
 'weatherDesc': [{'value': 'Partly cloudy'}],
 'precipMM': '0.1',
 'precipInches': '0.0',
 'humidity': '76',
 'visibility': '10',
 'visibilityMiles': '6',
 'pressure': '1023',
 'pressureInches': '31',
 'cloudcover': '45',
 'HeatIndexC': '17',
 'HeatIndexF': '62',
 'DewPointC': '12',
 'DewPointF': '54',
 'WindChillC': '17',
 'WindChillF': '62',
 'WindGustMiles': '1',
 'WindGustKmph': '2',
 'FeelsLikeC': '17',
 'FeelsLikeF': '62',
 'uvIndex': '5'}

In [17]:
df_2.isnull().sum()

date            0
astronomy       0
maxtempC        0
maxtempF        0
mintempC        0
mintempF        0
avgtempC        0
avgtempF        0
totalSnow_cm    0
sunHour         0
uvIndex         0
hourly          0
dtype: int64

In [18]:
df_2.shape

(3413, 12)

In [19]:
len(hourly_series)

3413

The second dictionary in the hourly column has the weather information for noon each day. This will be the source of the data we extract using the function below:

In [20]:
# Extract information from the hourly column that isn't represented already
def hourly_extractor(features, dataframe, hourly_series):
    """Extract needed information from the hourly column in the weather data, and add this data
    back to the overall dataset for later use.
    features - list of features to be extracted
    """
    # The 1 here extracts from the second dictionary
    for feature in features:
        dataframe[feature] = [hourly_series[num][1][feature] for num in range(len(hourly_series))]

In [21]:
# To make sure the original df isn't ruined 
df_3 = df_2.copy()

In [22]:
hourly_extractor(features=['humidity', 'precipInches', 'pressureInches', 'cloudcover', 'windspeedMiles',
                          'visibilityMiles'], dataframe=df_3, hourly_series=hourly_series)

In [23]:
df_3.tail()

,date,astronomy,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex,hourly,humidity,precipInches,pressureInches,cloudcover,windspeedMiles,visibilityMiles
29,2020-11-06,"[{'sunrise': '08:44 AM', 'sunset': '06:07 PM',...",11,52,6,42,8,47,0.0,9.6,3,"[{'time': '0', 'tempC': '7', 'tempF': '44', 'w...",52,0.0,31,1,9,6
30,2020-11-07,"[{'sunrise': '08:46 AM', 'sunset': '06:05 PM',...",15,59,7,44,12,53,0.0,9.6,4,"[{'time': '0', 'tempC': '7', 'tempF': '44', 'w...",71,0.0,31,5,5,6
31,2020-11-08,"[{'sunrise': '08:48 AM', 'sunset': '06:04 PM',...",16,61,9,49,13,56,0.0,6.5,4,"[{'time': '0', 'tempC': '10', 'tempF': '50', '...",72,0.0,31,81,7,6
32,2020-11-09,"[{'sunrise': '08:50 AM', 'sunset': '06:02 PM',...",16,61,11,52,14,57,0.0,9.5,4,"[{'time': '0', 'tempC': '12', 'tempF': '54', '...",70,0.0,31,40,6,6
33,2020-11-10,"[{'sunrise': '08:51 AM', 'sunset': '06:01 PM',...",15,58,12,53,13,56,0.0,4.8,3,"[{'time': '0', 'tempC': '12', 'tempF': '54', '...",79,0.0,31,83,4,6


In [24]:
# Drop the redundant and unnecesary columns - leave `hourly` just in case we want to extract more from it
df_3.drop(columns=['astronomy', 'maxtempC', 'mintempC', 'avgtempC'], inplace=True)

# # Save this version of the data
# df_3.to_csv('../data/weather_plus_hourly.csv', index=False)

In [25]:
# Now save a cleaner version to merge with the PV data
df_3.drop(columns='hourly', inplace=True)

df_3.to_csv('../data/weather_antwerp.csv', index=False)

Now that the weather data has been pulled in, it can be merged with the PV data based on the date column and then used to create a regression model. The merging of the datasets will be done in the [next notebook.
](./code2-data-wrangling-eda.ipynb)